In [10]:
import pygame
import random
import math
import heapq

# Initialize Pygame
pygame.init()

# Set up the display
width, height = 1520, 820
screen = pygame.display.set_mode((width, height), pygame.DOUBLEBUF)
pygame.display.set_caption("3D Robot")

# Load the background image
background_image = pygame.image.load("background.jpg").convert()
background_image = pygame.transform.scale(background_image, (screen.get_width(), screen.get_height()))

# Colors
WHITE = (255, 255, 255)
RED = (255, 0, 0)

# Robot class
class Robot(pygame.sprite.Sprite):
    def __init__(self, image_path):
        super().__init__()
        # Load the robot image
        self.image = pygame.image.load(image_path).convert_alpha()
        self.rect = self.image.get_rect()
        # radius used for collide_circle and to compute laser start from image edge
        self.radius = max(self.rect.width, self.rect.height) // 2
        self.direction = pygame.Vector2(1, 0)  # Initial direction
        self.move_delay = 0.0000000000000000001
        self.cleaned_dirts = 0
        self.total_dirts = 0
        self.font = pygame.font.Font(None, 36)
        self.message = ""
        self.target_dirt = None
        # start robot at center
        self.rect.center = (width // 2, height // 2)

    def update(self, dirts):
        # If there are dirts, pick the target using Dijkstra-based nearest selection
        if len(dirts) > 0:
            self.target_dirt = self.get_nearest_dirt_dijkstra(dirts)
            if self.target_dirt is not None:
                target_direction = pygame.Vector2(self.target_dirt.rect.center) - pygame.Vector2(self.rect.center)
                if target_direction.length() != 0:
                    target_direction.normalize_ip()
                    self.direction = target_direction

        # Move robot
        move_increment = 2
        self.rect.move_ip(self.direction.x * move_increment, self.direction.y * move_increment)

        # Check collisions with dirts (use list to safely modify group)
        for dirt in list(dirts):
            if pygame.sprite.collide_circle(self, dirt):
                # preserve original "move towards the dirt" behavior
                target_direction = pygame.Vector2(dirt.rect.center) - pygame.Vector2(self.rect.center)
                if target_direction.length() != 0:
                    target_direction.normalize_ip()
                    self.direction = target_direction

                # Clean the dirt (your existing condition)
                if dirt.color == RED:
                    dirts.remove(dirt)
                    self.cleaned_dirts += 1
                    if dirt is self.target_dirt:
                        self.target_dirt = None

        # Keep inside screen
        if not screen.get_rect().contains(self.rect):
            self.rect.clamp_ip(screen.get_rect())
            nearest_dirt = self.get_nearest_dirt_dijkstra(dirts)
            if nearest_dirt is not None:
                target_direction = pygame.Vector2(nearest_dirt.rect.center) - pygame.Vector2(self.rect.center)
                if target_direction.length() != 0:
                    target_direction.normalize_ip()
                    self.direction = target_direction
                self.target_dirt = nearest_dirt

        # Tiny move delay preserved
        pygame.time.wait(int(self.move_delay * 1000))

    def get_nearest_dirt_dijkstra(self, dirts):
        """
        Use Dijkstra on a fully connected graph of (robot + dirts).
        For a fully connected Euclidean graph this is equivalent to nearest-by-distance,
        but this uses Dijkstra as requested.
        """
        dirt_list = list(dirts)
        n = len(dirt_list)
        if n == 0:
            return None

        def dist_between_nodes(i, j):
            if i == 0:
                pos_i = pygame.Vector2(self.rect.center)
            else:
                pos_i = pygame.Vector2(dirt_list[i - 1].rect.center)
            if j == 0:
                pos_j = pygame.Vector2(self.rect.center)
            else:
                pos_j = pygame.Vector2(dirt_list[j - 1].rect.center)
            return (pos_i - pos_j).length()

        total_nodes = n + 1
        dist = [math.inf] * total_nodes
        dist[0] = 0
        visited = [False] * total_nodes
        heap = [(0, 0)]
        while heap:
            d, u = heapq.heappop(heap)
            if visited[u]:
                continue
            visited[u] = True
            for v in range(total_nodes):
                if u == v:
                    continue
                alt = d + dist_between_nodes(u, v)
                if alt < dist[v]:
                    dist[v] = alt
                    heapq.heappush(heap, (alt, v))

        # choose dirt node with minimal shortest-path distance
        min_distance = math.inf
        min_index = None
        for i in range(1, total_nodes):
            if dist[i] < min_distance:
                min_distance = dist[i]
                min_index = i

        if min_index is None:
            return None
        return dirt_list[min_index - 1]

    def display_message(self, message):
        self.message = message

    def draw(self, surface):
        # Draw the robot
        surface.blit(self.image, self.rect)

        # Draw a realistic laser beam from the robot's image edge towards target (if any)
        if self.target_dirt is not None:
            # compute normalized direction from robot center to target
            robot_center = pygame.Vector2(self.rect.center)
            target_center = pygame.Vector2(self.target_dirt.rect.center)
            dir_vec = target_center - robot_center
            if dir_vec.length() > 0:
                dir_norm = dir_vec.normalize()

                # compute start point at the robot image edge (uses radius)
                # add a small offset so laser starts just outside the robot image
                start_offset = self.radius + 2
                start_pos = robot_center + dir_norm * start_offset
                end_pos = target_center

                # Create a transparent surface for the laser glow (so we can use alpha)
                laser_surf = pygame.Surface((width, height), pygame.SRCALPHA)

                # Draw several lines with decreasing thickness and increasing alpha
                # Outer glows (soft)
                glow_steps = [
                    (12, 30),  # (thickness, alpha)
                    (8, 80),
                    (4, 160),
                    (2, 255),  # core bright line
                ]
                for thickness, alpha in glow_steps:
                    color = (RED[0], RED[1], RED[2], alpha)
                    # pygame.draw.line supports alpha on surfaces with SRCALPHA
                    pygame.draw.line(laser_surf, color, (int(start_pos.x), int(start_pos.y)),
                                     (int(end_pos.x), int(end_pos.y)), thickness)

                # Optionally draw a faint "ray" attenuation — draw a gradient by adding a semi-transparent
                # black along the end to simulate realistic falloff (subtle).
                # Draw a bright hit point on the dirt
                hit_radius = max(4, int(6 - (dir_vec.length() / 200)))  # smaller if far
                pygame.draw.circle(laser_surf, (255, 180, 180, 220), (int(end_pos.x), int(end_pos.y)), hit_radius)

                # Blit the laser surface onto main surface
                surface.blit(laser_surf, (0, 0))

        # draw message if present
        if self.message:
            message_surface = self.font.render(self.message, True, WHITE)
            message_rect = message_surface.get_rect(centerx=self.rect.centerx, y=self.rect.y - 20)
            surface.blit(message_surface, message_rect)


# Dirt class
class Dirt(pygame.sprite.Sprite):
    def __init__(self, image_path):
        super().__init__()
        self.radius = 10
        self.image = pygame.image.load(image_path).convert_alpha()
        self.rect = self.image.get_rect()
        self.color = RED
        self.reset()

    def reset(self):
        self.rect.center = (random.randint(0, width), random.randint(0, height))


# Create robot and dirts
robot = Robot("vc_1.png")
dirts = pygame.sprite.Group()
for _ in range(10):
    dirt = Dirt("d.png")
    dirts.add(dirt)
    robot.total_dirts += 1

clock = pygame.time.Clock()
running = True

# Main loop: run until either the user closes window OR cleaning completes.
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Update
    robot.update(dirts)

    # Draw
    screen.blit(background_image, (0, 0))
    dirts.draw(screen)
    robot.draw(screen)

    # Update message
    robot.display_message(f"Region of dirts Cleaned: {robot.cleaned_dirts}/{robot.total_dirts}")

    pygame.display.flip()
    clock.tick(60)

    # If cleaning is complete, break to waiting-for-quit state
    if robot.cleaned_dirts >= robot.total_dirts:
        break

# Cleaning finished — show final screen and allow user to quit (ESC or mouse click or window close)
finished = True
while finished:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            finished = False
        elif event.type == pygame.KEYDOWN:
            # ESC to quit
            if event.key == pygame.K_ESCAPE:
                finished = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            # any click quits
            finished = False

    # Draw final message
    screen.blit(background_image, (0, 0))
    # draw any remaining dirts (should be none) and robot
    dirts.draw(screen)
    robot.draw(screen)

    # show completion message
    big_font = pygame.font.Font(None, 72)
    small_font = pygame.font.Font(None, 36)
    text_surf = big_font.render("Cleaning complete!", True, WHITE)
    sub_surf = small_font.render("Press ESC, click or close window to quit.", True, WHITE)
    text_rect = text_surf.get_rect(center=(width // 2, height // 2 - 30))
    sub_rect = sub_surf.get_rect(center=(width // 2, height // 2 + 30))
    # Draw translucent background behind text for readability
    overlay = pygame.Surface((width, 140), pygame.SRCALPHA)
    overlay.fill((0, 0, 0, 140))
    overlay_rect = overlay.get_rect(center=(width // 2, height // 2))
    screen.blit(overlay, overlay_rect)
    screen.blit(text_surf, text_rect)
    screen.blit(sub_surf, sub_rect)

    pygame.display.flip()
    clock.tick(30)

pygame.quit()
